# Coincidencia de columnas desordenadas de Pandas con FuzzyWuzzy



![header](https://i.ibb.co/wLxk8nK/dinoheader-th.jpg)

En este notebook le mostraré cómo usar el paquete FuzzyWuzzy de Python para identificar coindidencias de  dos columnas de un dataset de Pandas utilizando la unción de la similitud de cadenas que nos aporta la liibrería FuzzyWuzzy

El resultado que deseamos obtener es que cada uno de los valores de la columna A coincida con el valor correspondiente más cercano en la columna B, entiendo como valor más cercano aquel que **a nivel probabilistico** es más probable en función del diseño del algoritmos utilizado


Para hacerlo he utilizado datos ficticios por lo que deberías poder adaptar esta pequeña utilizad al marco corporativo o empresarial al que desees aplicarlo  si finalmente decides que puede aportar automatización a tu proyecto 


Ahora, antes de abordar nuestro pequeño problema, esto es lo que puede esperar de este artículo:

     Casi a diario cualquier analista de datos suele trabajar con fuentes de datos muy heterogeneas en los que se encunetra  que la misma entidad a analizar (unidad de negocio, país, provincia, dirección, número de teléfono, email ,etc) se escribe de forma distinta, bien por la normalización heredada de las fuentes de origen , errores de tipeado manual en las introducciones cuando los datos no están referenciados a tablas o enumarados , etc 
     
     En este artículo, vamos a trabajar en hacer coincidir los nombres completos con los correos electrónicos. Sin embargo, puede aplicar las mismas técnicas para cualquier problema de coincidencia de cadenas que encuentre (la cadena es solo texto, en pocas palabras);

     Les dejaré este Jupyter Notebook en mi mi repositorio de Github por si quieren descagarlo y testear lo

     No será necesiario conocimientos amplios de Python, pero si sería deseable que estén familiarizdos con Pandas para el analisis de datos .
     


# Indice:


* [Introduccion](#intro) <br>
* [The problem](#theproblem)<br>
* [The packages](#thepackages)<br>
* [The data](#thedata)<br>
* [The "dumb but it kinda works" solution](#thedumb)<br>
* [The boring FuzzyWuzzy lecture](#theboring)<br>
* [The not so boring FuzzyWuzzy application](#thenotsoboring)<br>
* [The audit](#theaudit)<br>
* [The quest for the best FuzzyWuzzy scorer](#thequest)<br>
* [The battle of the scorers](#thebattle)<br>
* [The verdict](#theverdict)<br>
* [The appendix (code explanations)](#theappendix)
* [Parting notes](#parting)

# Instalación de paquetes utilizados

En primer lugar, importamos la libreria de Pandas :

In [2]:
import pandas as pd

Será necesario que tengas instalado para poder utilizar esta utilizar el paquete $FuzzyWuzzy$ y algunsos de sus modulos.


Puedes instalarlo con pip desde este Jupyter Notebook -- $pip install fuzzywuzzy$' ---   o también puedes instalarlo desde la consola de Anaconda si estas utilizando estra distribución. Debes instalarlo como `pip install fuzzywuzzy` y posteriormentte isntalar la utilizad para usar el algoritmo de --levenshtein` como  `$pip install python-Levenshtein$` 


In [7]:
from fuzzywuzzy import process, fuzz

La parte de visualización la abordaremos con  `matplotlib`:

In [20]:
import matplotlib.pyplot as plt

In [21]:
%%HTML
<style>.dataframe th, td:first-child{background:#3f577c;font-family:monospace;color:white;border:3px solid white;
text-align:left !important;}#codex{float:right;}</style>

In [22]:
th_props = [('background','#3f577c'), ('font-family','monospace'), ('color','#ff85d5'),('border','3px solid white'),
            ('text-align','left !important')]
styles = [dict(selector="th", props=th_props)]

Utilizaremos para este proyecto dos dataframes para realizar los matches :

* `hr` that contains employee IDs and full names, as maintained by the HR department ;
* `it` that contains only the emails extracted by the IT department.

Los dataset se encuentra en la carpeta datos del repositorio de Github que os anexo. Asegurate de cambiar los path a los que tengas establecidos en tu repositorio local sure to change the 'data/hr.csv' and 'data/it.csv' paths to where your data is placed in your machine (don't forget to get the data from this article's [Github repository](https://github.com/mbenaventc/FuzzyWuzzyLinkedin) :

In [25]:
calles = pd.read_excel('datos/callejero.xlsx')
customer=pd.read_excel('datos/customer.xlsx')


FileNotFoundError: [Errno 2] No such file or directory: 'datos/customer.xlsx'

In [24]:
hr.head(35)

,INDEX,CP,CM,CV,TV,NV
0,0,13,71,819,CL,FERRAZ - VILLAR
1,1,13,71,722,CL,SAN MIGUEL - VILLAR
2,2,13,71,860,TR,1 PINO
3,3,13,71,973,CL,5-SECTOR III
4,4,13,71,1,CL,ABAJO
5,5,13,71,901,PZ,ABENOJAR
6,6,13,71,145,CL,ADUANA
7,7,13,71,3,CL,AGUILERA
8,8,13,71,4,CL,AGUSTINA DE ARAGON
9,9,13,71,5,CL,ALAMEDA
